## RRN Heat Testing

Move A3 up and down 20 times using accelerations {0.2, 0.4, 0.6, 0.8, 1.0} $m/s^2$

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def fetchData(source):    
    #print("fetching data from path = ", source)
    data = pd.read_csv(source)
    return data

def appendTime(data):
    data['time'] = data['Loop time']/1000000;

    for i in range(1, data.shape[0], 1):
        data['time'][i] = data['time'][i] + data['time'][i-1] 

    return data

def addAccelerationForA3(data):
    data['A3_1 ACCSP'] = (data['A3_1 VELSP'] - data['A3_1 VELSP'].shift(1, fill_value=0))/(0.008)
    data['A3_1 ACCSP'] = data['A3_1 ACCSP'] + data['A3_1 ACCSP'].shift(1, fill_value=0)
    return data


def fetchA3Data(path, source, labels):
    data = fetchData(path + source)
    data = appendTime(data)
    data = addAccelerationForA3(data)
    labels.extend(['time', 'A3_1 ACCSP'])
    data = data[labels]
    return data 

def getDataForShadowing(dataArray, labels):
    tests = ['Acc_02', 'Acc_04', 'Acc_06', 'Acc_08', 'Acc_10']
    columnNames = [test+'_'+label for label inlabels for test in tests]

    dataFrame = pd.DataFrame(columns=columnNames)
    
    dataFrame['time'] = data['time']

    c = 0
    for index, data in enumerate(dataArray): #index is tests
        for label in labels:
            dataFrame[columnNames[c]] = data[label]
            c = c+1
    return 1

def printLabels(data):
    labels = data.columns.values
    print(labels)


def plot(data, labels, labelsOnSecondry, title, yLabel, secondYLabel):
    fig = make_subplots(rows=1, cols=1)
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    for label in labels:
        jointData = data[label]
     
        fig.add_trace(go.Scatter(x=data['time'], y=jointData, mode='lines', name=label), row=1, col=1, secondary_y=False)

    for label in labelsOnSecondry:
        jointData = data[label]
      
        fig.add_trace(go.Scatter(x=data['time'], y=jointData, mode='lines', name=label), row=1, col=1, secondary_y=True)

    # Update yaxis properties
    fig.update_yaxes(title_text=yLabel, row=1, col=1, secondary_y=False)
    fig.update_yaxes(title_text=secondYLabel, row=1, col=1, secondary_y=True)
    # Update xaxis properties
    fig.update_xaxes(title_text="time [s]", row=1, col=1)

    fig.update_layout(title_text=title, width=1550, height=1000)       
    fig.show()

def plotTorqueTurn(data, title):
    fig = make_subplots(rows=1, cols=1)
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    fig.add_trace(go.Scatter(x=data['turn'], y=data['torqueOnPipe'], mode='lines', name=label), row=1, col=1, secondary_y=False)

    # Update yaxis properties
    fig.update_yaxes(title_text='Torque', row=1, col=1)
    # Update xaxis properties
    fig.update_xaxes(title_text="Turn", row=1, col=1)

    fig.update_layout(title_text=title, width=1550, height=1000)       
    fig.show()

In [ ]:
path   = '/home/ep/dev/software/data-recorder-visualizator/'
source = [
    "RRN_T3/HeatTesting/GwLogData-18523-36074069_Acc02.csv",
    "RRN_T3/HeatTesting/GwLogData-18523-36825882_Acc04.csv",
    "RRN_T3/HeatTesting/GwLogData-18523-37521028_Acc06.csv",
    "RRN_T3/HeatTesting/GwLogData-18523-38080123_Acc08.csv",
    "RRN_T3/HeatTesting/GwLogData-18523-38800150_Acc10.csv"
]

sourceWarmup = ['RRN_T3/HeatTesting/GwLogData-18523-35374295_Warmup.csv']

#print(path + sourceWarmup[0])

### Plotting warm up data

In [ ]:
labels = ['A3_1 POSFB', 'A3_2 POSFB',
    'A3_1 VELFB', 'A3_2 VELFB',
    'A3_1 POSSP', 'A3_2 POSSP',
    'A3_1 VELSP', 'A3_2 VELSP',
    'A3_1 TORQUE', 'A3_2 TORQUE']
warmupData = fetchA3Data(path, sourceWarmup[0], labels)
#printLabels(warmupData)

LabelsLeft  = ['A3_1 TORQUE', 'A3_2 TORQUE']
LabelsRight = ['A3_1 VELFB', 'A3_2 VELFB', 'A3_1 VELSP', 'A3_2 VELSP','A3_1 ACCSP']
plot(warmupData, LabelsLeft, LabelsRight, "Warmup, speed factor = 50%, acceleration factor = 20%", "Torque [Nm]", "Velocity [m/s]")


### Getting data for the main test

In [ ]:
testData = []
for i, s in enumerate(source):
    labels = ['A3_1 POSFB', 'A3_2 POSFB',
                'A3_1 VELFB', 'A3_2 VELFB',
                'A3_1 POSSP', 'A3_2 POSSP',
                'A3_1 VELSP', 'A3_2 VELSP',
                'A3_1 TORQUE', 'A3_2 TORQUE']
    testData.append(fetchA3Data(path, s, labels))
    # print(i, ", ", s)

### Plotting test result for Acc = 0.2 $m/s^2$

In [ ]:
timeLimit = 40;
data = testData[0]
LabelsLeft  = ['A3_1 TORQUE', 'A3_2 TORQUE']
LabelsRight = ['A3_1 VELFB', 'A3_2 VELFB', 'A3_1 VELSP', 'A3_2 VELSP','A3_1 ACCSP']
plot(data[data['time'] < timeLimit], LabelsLeft, LabelsRight, "$Acc = 0.2 m/s^2$", "Torque [Nm]", "Velocity [m/s]")
del data

### Plotting test result for Acc = 0.4 $m/s^2$

In [ ]:
timeLimit = 40;
data = testData[1]
LabelsLeft  = ['A3_1 TORQUE', 'A3_2 TORQUE']
LabelsRight = ['A3_1 VELFB', 'A3_2 VELFB', 'A3_1 VELSP', 'A3_2 VELSP','A3_1 ACCSP']
plot(data[data['time'] < timeLimit], LabelsLeft, LabelsRight, "$Acc = 0.4 m/s^2$", "Torque [Nm]", "Velocity [m/s]")
del data

### Plotting test result for Acc = 0.6 $m/s^2$

In [ ]:
timeLimit = 40;
data = testData[2]
LabelsLeft  = ['A3_1 TORQUE', 'A3_2 TORQUE']
LabelsRight = ['A3_1 VELFB', 'A3_2 VELFB', 'A3_1 VELSP', 'A3_2 VELSP','A3_1 ACCSP']
plot(data[data['time'] < timeLimit], LabelsLeft, LabelsRight, "$Acc = 0.6 m/s^2$", "Torque [Nm]", "Velocity [m/s], Acceleartion[m/s2]")
#del data

In [ ]:
### Plotting test result for Acc = 0.8 $m/s^2$

In [ ]:
timeLimit = 40;
data = testData[3]
LabelsLeft  = ['A3_1 TORQUE', 'A3_2 TORQUE']
LabelsRight = ['A3_1 VELFB', 'A3_2 VELFB', 'A3_1 VELSP', 'A3_2 VELSP','A3_1 ACCSP']
plot(data[data['time'] < timeLimit], LabelsLeft, LabelsRight, "$Acc = 0.8 m/s^2$", "Torque [Nm]", "Velocity [m/s]")
del data

### Plotting test result for Acc = 1.0 $m/s^2$

In [ ]:
timeLimit = 40;
data = testData[4]
LabelsLeft  = ['A3_1 TORQUE', 'A3_2 TORQUE']
LabelsRight = ['A3_1 VELFB', 'A3_2 VELFB', 'A3_1 VELSP', 'A3_2 VELSP','A3_1 ACCSP']
plot(data[data['time'] < timeLimit], LabelsLeft, LabelsRight, "$Acc = 1.0 m/s^2$", "Torque [Nm]", "Velocity [m/s]")
del data

### Preparing for shadowing

In [ ]:
torques = getDataForShadowing(testData, ['A3_1 TORQUE', 'A3_2 TORQUE'])